In [363]:
import bs4, requests, re, bibtexparser

In [587]:
mode_layout = 'flat'
article_id = '337228'
expformat = 'bibtex'

domain = 'https://dl.acm.org/'
article = 'citation.cfm?id={0}&'.format(article_id)
parametrs = 'preflayout={0}'.format(mode_layout)
url = domain + article + parametrs
# https://dl.acm.org/downformats.cfm?id=2492591&parent_id=&expformat=bibtex
url_download_bibtex = domain + 'downformats.cfm?id={0}&parent_id=&expformat={1}'.format(article_id, expformat)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'}
file = open('test.html', 'wb')

print(url, url_download_bibtex, sep='\n')

https://dl.acm.org/citation.cfm?id=337228&preflayout=flat
https://dl.acm.org/downformats.cfm?id=337228&parent_id=&expformat=bibtex


In [588]:
response = requests.get(url=url, headers=headers)
soup = bs4.BeautifulSoup(response.text, 'lxml')
bibtex_file = open('bibtex.bib', 'w').write(requests.get(url=url_download_bibtex, headers=headers).text)
bibtex_file = open('bibtex.bib', 'r')
bibtex_dict = bibtexparser.load(bibtex_file).entries

In [589]:
article_data = {}

article_data['title'] = bibtex_dict[0].get('title', None)
article_data['doi'] = bibtex_dict[0].get('doi', None) 
article_data['year'] = bibtex_dict[0].get('year', None)

if bibtex_dict[0].get('series', None):
    article_data['venue'] = bibtex_dict[0].get('series', None).split()[0]
else:
    article_data['venue'] = None

article_data, bibtex_dict

({'doi': '10.1145/337180.337228',
  'title': 'Principled Design of the Modern Web Architecture',
  'venue': 'ICSE',
  'year': '2000'},
 [{'ENTRYTYPE': 'inproceedings',
   'ID': 'Fielding:2000:PDM:337180.337228',
   'acmid': '337228',
   'address': 'New York, NY, USA',
   'author': 'Fielding, Roy T. and Taylor, Richard N.',
   'booktitle': 'Proceedings of the 22Nd International Conference on Software Engineering',
   'doi': '10.1145/337180.337228',
   'isbn': '1-58113-206-9',
   'keywords': 'WWW, software architectural style, software architecture',
   'location': 'Limerick, Ireland',
   'numpages': '10',
   'pages': '407--416',
   'publisher': 'ACM',
   'series': "ICSE '00",
   'title': 'Principled Design of the Modern Web Architecture',
   'url': 'http://doi.acm.org/10.1145/337180.337228',
   'year': '2000'}])

In [590]:
divmain = soup.find('div', id='divmain')

authors_tags = divmain.find_all('td', {'style': 'padding-right:3px;', 'valign': 'top',  'nowrap': 'nowrap'})
authors = []

for author_tag in authors_tags:
    name = author_tag.text
    authors.append({'name': name.strip(),
                    'url': domain + author_tag.find('a')['href']})
    
article_data['authors'] = authors
article_data

{'authors': [{'name': 'Roy T. Fielding',
   'url': 'https://dl.acm.org/author_page.cfm?id=81100283787&coll=DL&dl=ACM&trk=0'},
  {'name': 'Richard N. Taylor',
   'url': 'https://dl.acm.org/author_page.cfm?id=81452609528&coll=DL&dl=ACM&trk=0'}],
 'doi': '10.1145/337180.337228',
 'title': 'Principled Design of the Modern Web Architecture',
 'venue': 'ICSE',
 'year': '2000'}

In [591]:
layout = soup.find('div', {'class': 'layout'})
flatbody = layout.find('div', {'class': 'flatbody'})

abstract = flatbody.text.strip()
article_data['abstract'] = abstract

article_data

{'abstract': 'The World Wide Web has succeeded in large part because its software architecture has been designed to meet the needs of an Internet-scale distributed hypermedia system. The modern Web architecture emphasizes scalability of component interactions, generality of interfaces, independent deployment of components, and intermediary components to reduce interaction latency, enforce security, and encapsulate legacy systems. In this paper, we introduce the Representational State Transfer (REST) architectural style, developed as an abstract model of the Web architecture to guide our redesign and definition of the Hypertext Transfer Protocol and Uniform Resource Identifiers. We describe the software engineering principles guiding REST and the interaction constraints chosen to retain those principles, contrasting them to the constraints of other architectural styles. We then compare the abstract model to the currently deployed Web architecture in order to elicit mismatches between th

In [607]:
td = soup.find('td', string='Conference').nextSibling.nextSibling
venue = td.strong.text.strip()

url_conference = td.a['href']

article_data['venue'] = {'name': venue,
                        'url': domain + url_conference}

article_data

{'abstract': 'The World Wide Web has succeeded in large part because its software architecture has been designed to meet the needs of an Internet-scale distributed hypermedia system. The modern Web architecture emphasizes scalability of component interactions, generality of interfaces, independent deployment of components, and intermediary components to reduce interaction latency, enforce security, and encapsulate legacy systems. In this paper, we introduce the Representational State Transfer (REST) architectural style, developed as an abstract model of the Web architecture to guide our redesign and definition of the Hypertext Transfer Protocol and Uniform Resource Identifiers. We describe the software engineering principles guiding REST and the interaction constraints chosen to retain those principles, contrasting them to the constraints of other architectural styles. We then compare the abstract model to the currently deployed Web architecture in order to elicit mismatches between th